# Practice notebook for regression analysis with dependent data in NHANES

This notebook will give you the opportunity to perform some analyses
using the regression methods for dependent data that we are focusing
on in this week of the course.

Enter the code in response to each question in the boxes labeled "enter your code here".
Then enter your responses to the questions in the boxes labeled "Type
Markdown and Latex".

This notebook is based on the NHANES case study notebook for
regression with dependent data.  Most of the code that you will need
to write below is very similar to code that appears in the case study
notebook.  You will need to edit code from that notebook in small ways
to adapt it to the prompts below.

To get started, we will use the same module imports and read the data
in the same way as we did in the case study:

In [18]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

url = "nhanes_2015_2016.csv"
da = pd.read_csv(url)

# Drop unused columns, drop rows with any missing values.
# added variables originally missing for this case study: BPXDI1
vars = ["BPXSY1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI",
        "SMQ020", "SDMVSTRA", "SDMVPSU", "BPXDI1"]
da = da[vars].dropna()

# This is the grouping variable
da["group"] = 10*da.SDMVSTRA + da.SDMVPSU

## Question 1: 

Build a **marginal linear model using GEE** for the first measurement of **diastolic blood pressure (`BPXDI1`)**, accounting for the **grouping variable** defined above.  **This initial model should have no covariates**, and will be used to **assess the ICC (intraclass correlation coefficient) of this blood pressure measure**.

In [2]:
# Marginal linear model
# Variance structure: Constant variance between subjects in a cluster/group
model = sm.GEE.from_formula("BPXDI1 ~ 1",
                            groups = "group",
                            cov_struct = sm.cov_struct.Exchangeable(),
                            data = da)
results = model.fit()
# results.summary()
print(results.cov_struct.summary())


The correlation between two observations in the same cluster is 0.031


__Q1a.__ What is the ICC for diastolic blood pressure?  What can you
  conclude by comparing it to the ICC for systolic blood pressure?

In [3]:
# Marginal linear model
# Variance structure: Constant variance between subjects in a cluster/group
model = sm.GEE.from_formula("BPXSY1 ~ 1",
                            groups = "group",
                            cov_struct = sm.cov_struct.Exchangeable(),
                            data = da)
results = model.fit()
# results.summary()
print(results.cov_struct.summary())


The correlation between two observations in the same cluster is 0.030


## Question 2: 

Take your model from question 1, and add gender, age, and BMI to it as covariates.

In [4]:
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male",
                                       2: "Female"})

In [5]:
# Marginal linear model
# Variance structure: Constant variance between subjects in a cluster/group
model = sm.GEE.from_formula("BPXDI1 ~ RIAGENDRx + RIDAGEYR + BMXBMI",
                            groups = "group",
                            cov_struct = sm.cov_struct.Exchangeable(),
                            data = da)
results = model.fit()
# results.summary()
print(results.cov_struct.summary())


The correlation between two observations in the same cluster is 0.030


__Q2a.__ What is the ICC for this model?  What can you conclude by comparing it to the ICC for the model that you fit in question 1?

## Question 3: 

Split the data into separate datasets for females and for males and fit two separate marginal linear models for diastolic blood pressure, one only for females, and one only for males.

In [6]:
da_male = da[da.RIAGENDRx == "Male"]
da_female = da[da.RIAGENDRx == "Female"]

In [7]:
i = 0

# Marginal linear model
# Variance structure: Constant variance between subjects in a cluster/group
for df in da_male, da_female:
    model = sm.GEE.from_formula("BPXSY1 ~ 1",
                                groups = "group",
                                cov_struct = sm.cov_struct.Exchangeable(),
                                data = df)
    results = model.fit()
    # results.summary()
    if(i == 0):
        print("For Males:")
    else:
        print("For Females:")
    print(results.cov_struct.summary())
    i += 1


For Males:
The correlation between two observations in the same cluster is 0.030
For Females:
The correlation between two observations in the same cluster is 0.030


__Q3a.__ What do you learn by comparing these two fitted models?

## Question 4: 

Using the entire data set, fit a **multilevel model** for diastolic blood pressure, predicted by age, gender, BMI, and educational attainment.  Include a random intercept for groups.

### Previous working which encountered index issues

In [34]:
da3 = da.copy()

In [35]:
da3

,BPXSY1,RIDAGEYR,RIAGENDR,RIDRETH1,DMDEDUC2,BMXBMI,SMQ020,SDMVSTRA,SDMVPSU,BPXDI1,group
0,128.0,62,1,3,5.0,27.8,1,125,1,70.0,1251
1,146.0,53,1,3,3.0,30.8,1,125,1,88.0,1251
2,138.0,78,1,3,3.0,28.8,1,131,1,46.0,1311
3,132.0,56,2,3,5.0,42.4,2,131,1,72.0,1311
4,100.0,42,2,4,4.0,20.3,2,126,2,70.0,1262
5,116.0,72,2,1,2.0,28.6,2,128,1,58.0,1281
6,110.0,22,1,4,4.0,28.0,1,128,2,70.0,1282
7,120.0,32,2,1,4.0,28.2,2,125,1,70.0,1251
9,178.0,56,1,4,3.0,33.6,2,126,2,116.0,1262
10,144.0,46,1,3,5.0,27.6,1,121,1,94.0,1211


In [36]:
da3["DMDEDUC2x"] = da3.DMDEDUC2.replace({1: "less_than_9",
                                        2: "9_11",
                                        3: "HS",
                                        4: "AA",
                                        5: "College",
                                        7: np.nan,
                                        9: np.nan})
da3["RIAGENDRx"] = da3.RIAGENDR.replace({1: "Male",
                                       2: "Female"})

In [37]:
da3.dropna().head()

,BPXSY1,RIDAGEYR,RIAGENDR,RIDRETH1,DMDEDUC2,BMXBMI,SMQ020,SDMVSTRA,SDMVPSU,BPXDI1,group,DMDEDUC2x,RIAGENDRx
0,128.0,62,1,3,5.0,27.8,1,125,1,70.0,1251,College,Male
1,146.0,53,1,3,3.0,30.8,1,125,1,88.0,1251,HS,Male
2,138.0,78,1,3,3.0,28.8,1,131,1,46.0,1311,HS,Male
3,132.0,56,2,3,5.0,42.4,2,131,1,72.0,1311,College,Female
4,100.0,42,2,4,4.0,20.3,2,126,2,70.0,1262,AA,Female


In [38]:
# center the age variable
da3["age_cen"] = da3.groupby("group").RIDAGEYR.transform(lambda x: x - x.mean())

In [39]:
#https://www.statsmodels.org/dev/generated/statsmodels.regression.mixed_linear_model.MixedLM.from_formula.html
# re_formula = None => The default gives a random intercept for each group.
# vc_formula = None => formula for the component with variance parameter  

# Multi-level model
# Random effect for intercept
# model = sm.MixedLM.from_formula("BPXDI1 ~ age_cen + RIAGENDRx + BMXBMI + C(DMDEDUC2x)",
#                             groups = "group",
#                             data = da)
# re_formula = "1 + age_cen"
# IndexError: index 5101 is out of bounds for axis 1 with size 5100
# ValueError: Shape mismatch between endog/exog and extra 2d arrays given to model.


# subset into a smaller dataframe with only the required cols
vars3 = ["age_cen", "RIAGENDRx", "DMDEDUC2x", "BMXBMI",
        "group", "BPXDI1"]
da3 = da3[vars3]
da3.dropna()


,age_cen,RIAGENDRx,DMDEDUC2x,BMXBMI,group,BPXDI1
0,6.671429,Male,College,27.8,1251,70.0
1,-2.328571,Male,HS,30.8,1251,88.0
2,23.553763,Male,HS,28.8,1311,46.0
3,1.553763,Female,College,42.4,1311,72.0
4,-7.219653,Female,AA,20.3,1262,70.0
5,16.322727,Female,9_11,28.6,1281,58.0
6,-25.431818,Male,AA,28.0,1282,70.0
7,-23.328571,Female,AA,28.2,1251,70.0
9,6.780347,Male,HS,33.6,1262,116.0
10,-0.195652,Male,College,27.6,1211,94.0


In [44]:
# over here, we see that even after using dropna() on the da3 dataframe, someone 2 records with 'NaN' in the 'DMDEDUC2x' column
# still remained in the dataset
da3.loc[drop_cases,:]

,age_cen,RIAGENDRx,DMDEDUC2x,BMXBMI,group,BPXDI1
894,16.396226,Female,NaN,26.6,1192,60.0
4964,30.780347,Male,NaN,33.9,1262,68.0


In [47]:
da4 = da3.dropna()

In [48]:
# working as expected => the records (with a NaN in DMDEDUC2x) with those 2 indexes were dropped
da4.loc[drop_cases,:]

KeyError: "None of [Int64Index([894, 4964], dtype='int64')] are in the [index]"

In [49]:
print("Dataframe 2 from the 'right' working below as the shape :", da2.shape)
print("Dataframe 3 from the 'wrong' working above has the shape :", da3.shape)
print("Dataframe 4 from the working above after assigning df3 to df4 when performing dropna() has the shape :", da4.shape)

Dataframe 2 from the 'right' working below as the shape : (5100, 6)
Dataframe 3 from the 'wrong' working above has the shape : (5102, 6)
Dataframe 4 from the working above after assigning df3 to df4 when performing dropna() has the shape : (5100, 6)


### New working which worked

In [ ]:
# therefore we need to use the approach below which extracts out the rows we dont want
# retrieve their index #s
# drop their rows from the dataframe

# then continue with the remaining steps we performed 
# - to recode the variables
# - to fit a multi-level model

In [15]:
#### DROP CASES WITH LEVEL 7 or 9 for DMDEDUC2
drop_cases = da[(da['DMDEDUC2'] == 7) | (da['DMDEDUC2'] == 9)].index
da.drop(drop_cases, inplace = True)

### RECODE THE REMAINING LEVELS IN DMDEDUC2
da["DMDEDUC2x"] = da.DMDEDUC2.replace({1: "less_than_9",
                                        2: "9_11",
                                        3: "HS",
                                        4: "AA",
                                        5: "College"})

# center the age variable
da["age_cen"] = da.groupby("group").RIDAGEYR.transform(lambda x: x - x.mean())


# subset into a smaller dataframe with only the required cols
vars2 = ["age_cen", "RIAGENDRx", "DMDEDUC2x", "BMXBMI", "group", "BPXDI1"]
da2 = da[vars2]
da2.dropna()
da2 = da2.reset_index(drop = True)

In [16]:
### REMOVE C() FROM THE MODEL STATEMENT as the model already understands that we have categorical labels on "DMDEDUC2x"
model = sm.MixedLM.from_formula("BPXDI1 ~ age_cen + RIAGENDRx + BMXBMI + DMDEDUC2x",
                            groups = "group",
                            data = da2)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
               Mixed Linear Model Regression Results
===================================================================
Model:                MixedLM    Dependent Variable:    BPXDI1     
No. Observations:     5100       Method:                REML       
No. Groups:           30         Scale:                 154.2358   
Min. group size:      105        Likelihood:            -20112.8400
Max. group size:      226        Converged:             Yes        
Mean group size:      170.0                                        
-------------------------------------------------------------------
                         Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------
Intercept                63.496    1.017 62.432 0.000 61.502 65.489
RIAGENDRx[T.Male]         2.754    0.351  7.850 0.000  2.066  3.441
DMDEDUC2x[T.AA]          -0.204    0.605 -0.337 0.736 -1.390  0.982
DMDEDUC2x[T.College]      0.224    0.630  0.355 0.723 -1.012  1.459
DMDEDUC2x[T.HS]          -0.873    0.633 -1.379 0.168 -2.113  0.367
DMDEDUC2x[T.less_than_9] -0.752    0.726 -1.035 0.301 -2.176  0.672
age_cen                  -0.037    0.010 -3.634 0.000 -0.057 -0.017
BMXBMI                    0.186    0.026  7.275 0.000  0.136  0.236
group Var                 4.489    0.114                           
===================================================================

"""

__Q4a.__ How would you describe the strength of the clustering in this analysis?

In [50]:
np.sqrt(2*4.489)

2.996331089849718

__Q4b:__ Include a random intercept for age, and describe your findings.

In [51]:
model = sm.MixedLM.from_formula("BPXDI1 ~ age_cen + RIAGENDRx + BMXBMI + DMDEDUC2x",
                            groups = "group",
                            re_formula = "age_cen",
                            data = da2)
results = model.fit()
results.summary()

/opt/conda/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2045: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
               Mixed Linear Model Regression Results
===================================================================
Model:                MixedLM    Dependent Variable:    BPXDI1     
No. Observations:     5100       Method:                REML       
No. Groups:           30         Scale:                 153.1751   
Min. group size:      105        Likelihood:            -20106.4776
Max. group size:      226        Converged:             Yes        
Mean group size:      170.0                                        
-------------------------------------------------------------------
                         Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------
Intercept                63.518    1.016 62.520 0.000 61.527 65.509
RIAGENDRx[T.Male]         2.755    0.350  7.870 0.000  2.069  3.441
DMDEDUC2x[T.AA]          -0.218    0.604 -0.361 0.718 -1.402  0.965
DMDEDUC2x[T.College]      0.211    0.629  0.336 0.737 -1.022  1.445
DMDEDUC2x[T.HS]          -0.856    0.631 -1.355 0.175 -2.094  0.382
DMDEDUC2x[T.less_than_9] -0.710    0.727 -0.976 0.329 -2.135  0.716
age_cen                  -0.033    0.015 -2.175 0.030 -0.063 -0.003
BMXBMI                    0.185    0.026  7.256 0.000  0.135  0.235
group Var                 4.524    0.115                           
group x age_cen Cov       0.039    0.003                           
age_cen Var               0.004    0.000                           
===================================================================

"""

# Extra tests from other exercises on p-value
https://support.minitab.com/en-us/minitab/18/help-and-how-to/statistics/basic-statistics/supporting-topics/basics/manually-calculate-a-p-value/

In [45]:
# p-value for 1-sided test
import scipy.stats

scipy.stats.t.cdf(0.278,df=97)

0.6091979711917032

In [46]:
# p-value for 2-sided test
import scipy.stats

scipy.stats.t.cdf(-0.278,df=97)*2

# 0.7816040576165936

0.7816040576165936